In [5]:
import pandas as pd
import os
os.chdir(r"/Users/apple/Documents/Credit_Card_Churn_Model_Local/credit-card-churn")
#Initialising Dataset
data = pd.read_csv("/Users/apple/Documents/Credit_Card_Churn_Model_Local/credit-card-churn/cleaned_credit_card_churn_numeric_feature_selection.csv") 